# Libraries

In [25]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [26]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\in\wc_goals.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,referee_nationality,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,Group A,18:00,Germany,Costa Rica,4–2,Allianz Arena,Munich,66000,Horacio Elizondo,Argentina,...,6,0,4,2,0,0,0,0,NaN,9 June 2006
1,Group A,18:00,Germany,Costa Rica,4–2,Allianz Arena,Munich,66000,Horacio Elizondo,Argentina,...,17,0,4,2,0,0,0,0,NaN,9 June 2006
2,Group A,18:00,Germany,Costa Rica,4–2,Allianz Arena,Munich,66000,Horacio Elizondo,Argentina,...,61,0,4,2,0,0,0,0,NaN,9 June 2006
3,Group A,18:00,Germany,Costa Rica,4–2,Allianz Arena,Munich,66000,Horacio Elizondo,Argentina,...,87,0,4,2,0,0,0,0,NaN,9 June 2006
4,Group A,18:00,Germany,Costa Rica,4–2,Allianz Arena,Munich,66000,Horacio Elizondo,Argentina,...,12,0,4,2,0,0,0,0,NaN,9 June 2006


# Clean and transfrom variables

## time

In [27]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## date

In [28]:
# Convert 'short_date' column to datetime if it's not already
df['short_date'] = pd.to_datetime(df['short_date'])

# Extract the year from 'short_date' and create a new column 'year'
df['year'] = df['short_date'].dt.year

# Extract relevant columns

In [29]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
0,NaN,Group A,Germany,Costa Rica,Germany,6,NaT,18:00,4–2
1,NaN,Group A,Germany,Costa Rica,Germany,17,NaT,18:00,4–2
2,NaN,Group A,Germany,Costa Rica,Germany,61,NaT,18:00,4–2
3,NaN,Group A,Germany,Costa Rica,Germany,87,NaT,18:00,4–2
4,NaN,Group A,Germany,Costa Rica,Costa Rica,12,NaT,18:00,4–2


# Recreate Leauge Table after first two matchdays

### Filter out the matches from the last match day in the goals_df DataFrame.

In [30]:
# Step 1: Find the last match date for each tournament and group
last_dates = goals_df.groupby(['year', 'stage'])['short_date'].max().reset_index()

# Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
# Performing a left merge and keeping only rows not present in the last_dates
goals_before_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='left', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Create the dataset with only the last match day games (goals_last_day)
goals_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='inner')

# Step 4: Sort the DataFrame by 'goal_minute'
goals_last_day_sorted = goals_last_day.sort_values(by=['goal_minute'], ascending=True)

# Ensure that goals_last_day_sorted has no duplicates
goals_last_day_sorted = goals_last_day_sorted.drop_duplicates()

In [31]:
goals_last_day_sorted

,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
65,2022.0,Group F,Canada,Morocco,Morocco,4,2022-12-01,18:00,1–2
37,2018.0,Third place play-off,Belgium,England,Belgium,4,2018-07-14,17:00,2–0
77,2022.0,Group H,South Korea,Portugal,Portugal,5,2022-12-02,18:00,2–1
78,2022.0,Third place play-off,Croatia,Morocco,Croatia,7,2022-12-17,18:00,2–1
80,2022.0,Third place play-off,Croatia,Morocco,Morocco,9,2022-12-17,18:00,2–1
...,...,...,...,...,...,...,...,...,...
25,2018.0,Group E,Switzerland,Costa Rica,Costa Rica,93,2018-06-27,21:00,2–2
6,2018.0,Group B,Iran,Portugal,Iran,93,2018-06-25,21:00,1–1
4,2018.0,Group A,Saudi Arabia,Egypt,Saudi Arabia,95,2018-06-25,17:00,2–1
50,2022.0,Group C,Saudi Arabia,Mexico,Saudi Arabia,95,2022-11-30,22:00,1–2


## create df  of aggregate data for each match

In [32]:
# Initialize an empty list to store the results
results = []

# Iterate over each match in the dataset
for match_id, group in goals_before_last_day.groupby(['year', 'stage', 'home_team', 'away_team']):
    # Initialize goals_home and goals_away for each match
    goals_home = 0
    goals_away = 0
    
    # Extract the local_time and the score from the first row in the group
    local_time = group.iloc[0]['local_time']
    score = group.iloc[0]['score']  # Assuming score is stored with a longer dash like '2–0'
    
    # Extract short_date from local_time (assuming local_time is a datetime object or string convertible to date)
    short_date = pd.to_datetime(local_time).date()  # Convert local_time to a short date (YYYY-MM-DD)
    
    # Loop through each row in the group to count goals for home and away teams
    for _, row in group.iterrows():
        # Check if the scorer's nationality matches the home team
        if row['home_team'] == row['scorer_nationality']:
            goals_home += 1
        # If the scorer's nationality matches the away team, increment goals_away
        elif row['away_team'] == row['scorer_nationality']:
            goals_away += 1

    # Calculate the expected score from goals_home and goals_away (using a standard hyphen)
    calculated_score = f"{goals_home}-{goals_away}"
    
    # Normalize both scores by replacing any long dash (en-dash or em-dash) with a standard hyphen
    normalized_score = score.replace("–", "-").replace("—", "-")
    
    # Check if the normalized score matches the calculated score
    score_match = normalized_score == calculated_score
    
    # Append the results to the list, including local_time, short_date, and score match flag
    results.append({
        'year': match_id[0],
        'stage': match_id[1],
        'home_team': match_id[2],
        'away_team': match_id[3],
        'local_time': local_time,
        'short_date': short_date,  # New column for the short date
        'goals_home': goals_home,
        'goals_away': goals_away,
        'original_score': score,  # Keep the original score with long dash
        'calculated_score': calculated_score,
        'score_match': score_match  # True if score matches, False otherwise
    })

# Convert the list into a DataFrame
agg_goals_before_last_day = pd.DataFrame(results)

# Add a new column 'won' based on the comparison of goals_home and goals_away
agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
    lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
    axis=1
)

In [33]:
agg_goals_before_last_day

,year,stage,home_team,away_team,local_time,short_date,goals_home,goals_away,original_score,calculated_score,score_match,won
0,2018.0,Group A,Egypt,Uruguay,17:00,2024-11-01,0,1,0–1,0-1,True,-1
1,2018.0,Group A,Russia,Egypt,21:00,2024-11-01,3,1,3–1,3-1,True,1
2,2018.0,Group A,Russia,Saudi Arabia,18:00,2024-11-01,5,0,5–0,5-0,True,1
3,2018.0,Group A,Uruguay,Saudi Arabia,18:00,2024-11-01,1,0,1–0,1-0,True,1
4,2018.0,Group B,Iran,Spain,21:00,2024-11-01,0,1,0–1,0-1,True,-1
5,2018.0,Group B,Morocco,Iran,18:00,2024-11-01,0,1,0–1,0-1,True,-1
6,2018.0,Group B,Portugal,Morocco,15:00,2024-11-01,1,0,1–0,1-0,True,1
7,2018.0,Group B,Portugal,Spain,21:00,2024-11-01,3,3,3–3,3-3,True,0
8,2018.0,Group C,Denmark,Australia,16:00,2024-11-01,1,1,1–1,1-1,True,0
9,2018.0,Group C,France,Australia,13:00,2024-11-01,2,1,2–1,2-1,True,1


## aggregate data for home and away games

In [34]:
def calculate_points(results, years, win_result):
    points = []
    for result, year in zip(results, years):
        if result == win_result:  # Win condition (1 for home, -1 for away)
            points.append(2 if year <= 1994 else 3)
        elif result == 0:  # Draw condition
            points.append(1)
        else:  # Loss condition
            points.append(0)
    return sum(points)

In [35]:
# Step 1: Home games aggregation with match count and conditional points based on year
home_games = agg_goals_before_last_day.groupby(['year', 'stage', 'home_team']).agg(
    goals_scored=('goals_home', 'sum'),
    goals_conceded=('goals_away', 'sum'),
    points_home=('won', lambda x: calculate_points(x, agg_goals_before_last_day.loc[x.index, 'year'], 1)),
    match_count_home=('home_team', 'count')
).reset_index()


In [36]:
display(home_games.head())

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home
0,2018.0,Group A,Egypt,0,1,0,1
1,2018.0,Group A,Russia,8,1,6,2
2,2018.0,Group A,Uruguay,1,0,3,1
3,2018.0,Group B,Iran,0,1,0,1
4,2018.0,Group B,Morocco,0,1,0,1


In [37]:
# Filter for year == 2021 and stage == 'Group F'
group_f_2021 = home_games[(home_games['year'] == 2022) & (home_games['stage'] == 'Group E')]
group_f_2021

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home
34,2022.0,Group E,Germany,1,2,0,1
35,2022.0,Group E,Japan,0,1,0,1
36,2022.0,Group E,Spain,8,1,4,2


In [38]:
# Step 2: Away games aggregation with match count and conditional points based on year

away_games = agg_goals_before_last_day.groupby(['year', 'stage', 'away_team']).agg(
    goals_scored=('goals_away', 'sum'),
    goals_conceded=('goals_home', 'sum'),
    points_away=('won', lambda x: calculate_points(x, agg_goals_before_last_day.loc[x.index, 'year'], -1)),
    match_count_away=('away_team', 'count')
).reset_index()

In [39]:
display(away_games.head())

,year,stage,away_team,goals_scored,goals_conceded,points_away,match_count_away
0,2018.0,Group A,Egypt,1,3,0,1
1,2018.0,Group A,Saudi Arabia,0,6,0,2
2,2018.0,Group A,Uruguay,1,0,3,1
3,2018.0,Group B,Iran,1,0,3,1
4,2018.0,Group B,Morocco,0,1,0,1


## aggregate data after first two matches following 

### FIFA criteria (first aggregate level, then h2h) 

In [40]:

def fifa_before_last(home_games, away_games, agg_goals_before_last_day):
    # Step 1: Merge home_games and away_games on year, stage, and home_team with away_team
    all_games_before_last = pd.merge(
        home_games,
        away_games,
        left_on=['year', 'stage', 'home_team'],
        right_on=['year', 'stage', 'away_team'],
        how='outer',
        suffixes=('_home', '_away')
    )

    # Step 2: For teams that only appear in home_games, assign home_team to team and copy relevant columns
    all_games_before_last['team'] = all_games_before_last['home_team'].fillna(all_games_before_last['away_team'])

    # Step 3: For goals_scored, goals_conceded, and points, handle missing values
    all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
    all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
    all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)

    # Step 4: Add goals_difference column
    all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

    # Step 5: Sum match_count_home and match_count_away to get total match count for each team
    all_games_before_last['total_matches'] = all_games_before_last['match_count_home'].fillna(0) + all_games_before_last['match_count_away'].fillna(0)

    # Step 6: Adjust points if total_matches is 1 (indicating a likely 0-0 draw)
    all_games_before_last.loc[all_games_before_last['total_matches'] == 1, 'points'] += 1

    # Step 7: Drop unnecessary columns used in the merge process
    all_games_before_last = all_games_before_last[['year', 'stage', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']]

    # Step 8: Initial sort by year, stage, points, goals_difference, and goals_scored within each year and stage
    all_games_before_last = all_games_before_last.sort_values(
        by=['year', 'stage', 'points', 'goals_difference', 'goals_scored'],
        ascending=[True, True, False, False, False]
    ).reset_index(drop=True)

    # Step 9: Initialize a new column for tie-break results, which will store either the tie-break result or 'no need'
    all_games_before_last['tiebreaker'] = 'no need'

    # Step 10: Define the function to check the tie-breaker
    def check_tiebreaker(row1, row2, agg_data):
        """
        Check the tie-breaker based on the head-to-head match result from agg_goals_before_last_day.
        Return the team that won (if any) or 'tie'.
        """
        # Look for the head-to-head match between the two teams in agg_goals_before_last_day
        match = agg_data[((agg_data['home_team'] == row1['team']) & (agg_data['away_team'] == row2['team'])) |
                         ((agg_data['home_team'] == row2['team']) & (agg_data['away_team'] == row1['team']))]

        if not match.empty:
            if match.iloc[0]['won'] == 1:
                return row1['team']  # Home team won
            elif match.iloc[0]['won'] == -1:
                return row2['team']  # Away team won
            else:
                return 'tie'  # It's a draw
        else:
            return 'no result'  # No match found

    # Step 11: Loop through the sorted dataframe and apply the tie-breaker if needed
    for i in range(len(all_games_before_last) - 1):
        row1 = all_games_before_last.iloc[i]
        row2 = all_games_before_last.iloc[i + 1]

        # Check if the two rows have identical values for points, goal difference, and goals scored
        if (row1['points'] == row2['points'] and
            row1['goals_difference'] == row2['goals_difference'] and
            row1['goals_scored'] == row2['goals_scored']):

            # Apply the tie-breaker by checking the head-to-head match result
            tiebreak_result = check_tiebreaker(row1, row2, agg_goals_before_last_day)

            # Store the tie-breaker result for both teams involved
            all_games_before_last.at[i, 'tiebreaker'] = tiebreak_result
            all_games_before_last.at[i + 1, 'tiebreaker'] = tiebreak_result

    # Step 12: Add group standing by ranking teams within each stage based on points, goal difference, and goals scored
    all_games_before_last['standing'] = all_games_before_last.groupby(['year', 'stage']).apply(
        lambda x: x.sort_values(by=['points', 'goals_difference', 'goals_scored'], ascending=[False, False, False])
    ).reset_index(drop=True).groupby(['year', 'stage']).cumcount() + 1

    # Convert goals_scored, goals_conceded, points, goals_difference, and total_matches to integers
    all_games_before_last[['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']] = all_games_before_last[
        ['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']].astype(int)

    return all_games_before_last


In [41]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = fifa_before_last(home_games, away_games, agg_goals_before_last_day)


C:\Users\aldi\AppData\Local\Temp\ipykernel_24060\255372799.py:79: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_before_last['standing'] = all_games_before_last.groupby(['year', 'stage']).apply(


In [42]:
# Filter specific group
filtered_games = all_games_before_last[(all_games_before_last['stage'] == 'Group E') & 
                                       (all_games_before_last['year'] == 2022)]

# Display the filtered dataset
display(filtered_games)

,year,stage,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,standing
48,2022.0,Group E,Spain,8,1,4,7,2,no need,1
49,2022.0,Group E,Japan,2,2,3,0,2,no need,2
50,2022.0,Group E,Costa Rica,1,7,3,-6,2,no need,3
51,2022.0,Group E,Germany,2,3,1,-1,2,no need,4


# Recreate league table after last match day

### fifa criteria

In [43]:
def fifa_final(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    # Step 1: Filter the data for the specific year and stage
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['year'] == year) & 
        (all_games_before_last['stage'] == stage)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['year'] == year) & 
        (goals_last_day_sorted['stage'] == stage)
    ]

    # Step 2: Initialize columns for tracking team performance
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']

    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']
    group_goals_tracking['last_game_standing'] = 0

  # Initialize position counters based on initial standings
    group_goals_tracking['1st'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 1 else 0)
    group_goals_tracking['2nd'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 2 else 0)
    group_goals_tracking['3rd'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 3 else 0)
    group_goals_tracking['4th'] = group_goals_tracking['before_last_game_standing'].apply(lambda x: 1 if x == 4 else 0)

    group_goals_tracking['changes'] = 0  # Initialize this but will be redefined later as the sum of 1st, 2nd, 3rd, 4th
    group_goals_tracking['tied'] = False  # Initialize a flag to track tied teams
    group_goals_tracking['tied_won'] = 0  # Initialize to track if the team won a tie-breaker match

    # Step 3: Sort group_goals_last_day by goal_minute in ascending order
    group_goals_last_day = group_goals_last_day.sort_values(by='goal_minute', ascending=True)

    # Print the year, group_name, and standings before starting the loop for last match goals
    print(f"\n=== Initial Standings for Year {year}, Group {stage} Before Last Match Goals ===\n")
    display_columns = ['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 
                       'total_goal_difference', 'before_last_game_points', 'before_last_game_standing']
    print(group_goals_tracking[display_columns].to_string(index=False))
    print("\n====================================================\n")

    # Step 4: Iterate through the sorted and filtered last match goals and update the goals_tracking table
    previous_standings = group_goals_tracking['last_game_standing'].copy()
    first_iteration = True  # Variable to track the first iteration

    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home_team']
        away_team = goal['away_team']
        player_team = goal['scorer_nationality']

        # Print goal information for each goal
        print(f"Analyzing goal: {goal['goal_minute']} minute, Player team: {player_team}, Home: {home_team}, Away: {away_team}")

        # Update the goals based on who scored the goal (no own goal condition anymore)
        if player_team == home_team:
            # Home team scored, update home scored and away conceded
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
        elif player_team == away_team:
            # Away team scored, update away scored and home conceded
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1

        # Step 5: Update total_goals_scored, total_goals_conceded, and total_goal_difference
        group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored'] + group_goals_tracking['last_game_goals_scored']
        group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded'] + group_goals_tracking['last_game_goals_conceded']
        group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Step 6: Assign points for the last game dynamically after each goal
        for i, row in group_goals_tracking.iterrows():
            if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
                # Check if the year is <= 1994, award 2 points for a win, else award 3 points
                if row['year'] <= 1994:
                    group_goals_tracking.loc[i, 'last_game_points'] = 2  # Win before or during 1994
                else:
                    group_goals_tracking.loc[i, 'last_game_points'] = 3  # Win after 1994
            elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 1  # Draw
            else:
                group_goals_tracking.loc[i, 'last_game_points'] = 0  # Loss

        # Step 7: Update total points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']

        # Step 8: Mark teams that are tied
        group_goals_tracking['tied'] = group_goals_tracking.duplicated(subset=['total_points', 'total_goal_difference', 'total_goals_scored'], keep=False)

        # Reset `tied_won` to 0 for all teams
        group_goals_tracking['tied_won'] = 0

        # Step 8b: Calculate tied_won only for tied teams
        tied_teams = group_goals_tracking[group_goals_tracking['tied']]

        if not tied_teams.empty:
            # Iterate over tied teams to resolve standings using head-to-head results
            for index, row in tied_teams.iterrows():
                team1 = row['team']
                team1_index = index

                # Look for other teams tied with this team
                for other_index, other_row in tied_teams[tied_teams.index != index].iterrows():
                    team2 = other_row['team']
                    team2_index = other_index

                    # Check if these two teams played against each other in agg_goals_before_last_day
                    match = agg_goals_before_last_day[
                        ((agg_goals_before_last_day['home_team'] == team1) & (agg_goals_before_last_day['away_team'] == team2)) |
                        ((agg_goals_before_last_day['home_team'] == team2) & (agg_goals_before_last_day['away_team'] == team1))
                    ]

                    if not match.empty:
                        match_result = match.iloc[0]['won']  # Assume the 'won' column holds 1 for home win, -1 for away win, 0 for draw

                        # Resolve the tie using the match result
                        if match_result == 1:
                            # Home team won
                            if match.iloc[0]['home_team'] == team1:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 1
                            else:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 1
                        elif match_result == -1:
                            # Away team won
                            if match.iloc[0]['away_team'] == team1:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 1
                            else:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 1
                        elif match_result == 0:
                            # Draw, both teams get the same standing
                            group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 0
                            group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 0

        # Step 8c: Sort teams by total points, goal difference, goals scored, and tied_won
        group_goals_tracking = group_goals_tracking.sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored', 'tied_won'],
                                                                ascending=[False, False, False, False])

        # Step 9: Assign standings based on the sorting and tie resolution
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1

        # Step 10: Track changes and update standing positions after each goal is processed
        for i, row in group_goals_tracking.iterrows():
            team = row['team']

            if first_iteration:
                # Skip the update if the standing did not change
                if row['before_last_game_standing'] == row['last_game_standing']:
                    continue  # Skip updating the counters
                else:
                    # Update the position counters since standing has changed
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

                # Disable first iteration flag after the first goal
                first_iteration = False
            else:
                # Track the standing position counters only if the position differs from the previous state
                if row['last_game_standing'] != previous_standings[i]:  
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

        # Update previous standings after each goal
        previous_standings = group_goals_tracking['last_game_standing'].copy()

        # Step 11: Calculate changes as the sum of 1st, 2nd, 3rd, and 4th
        group_goals_tracking['changes'] = group_goals_tracking[['1st', '2nd', '3rd', '4th']].sum(axis=1)

        # Step 12: Print the updated group_goals_tracking after processing each goal
        print("\n=== Updated Standings After This Goal ===\n")
        display_columns = ['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 
                           'total_goal_difference', 'last_game_points', 'last_game_standing', 
                           'changes', '1st', '2nd', '3rd', '4th', 'tied', 'tied_won']
        print(group_goals_tracking[display_columns].to_string(index=False))
        print("\n========================================\n")

    # Step 13: Return the final DataFrame
    return group_goals_tracking


In [44]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year and stage
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    result = fifa_final(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[['year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]



=== Initial Standings for Year 2018.0, Group Group A Before Last Match Goals ===

        team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_points  before_last_game_standing
      Russia             6                   8                     1                      7                        6                          1
     Uruguay             6                   2                     0                      2                        6                          2
       Egypt             0                   1                     4                     -3                        0                          3
Saudi Arabia             0                   0                     6                     -6                        0                          4


Analyzing goal: 10 minute, Player team: Uruguay, Home: Uruguay, Away: Russia

=== Updated Standings After This Goal ===

        team  total_points  total_goals_scored  total_goals_conceded  tota

In [45]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\tb_wc_fifa_wiki.xlsx'
changes_df_euro.to_excel(file_path, index=False)

# group composition tracking

In [46]:
def track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    # Step 1: Filter the data for the specific year and stage
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['year'] == year) & 
        (all_games_before_last['stage'] == stage)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['year'] == year) & 
        (goals_last_day_sorted['stage'] == stage)
    ]

    # Initialize columns for team performance and standings
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']
    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']
    group_goals_tracking['last_game_standing'] = 0

    # Define top standings limit based on the year
    top_standings_limit = 3 if year <= 1994 else 2

    # Determine the initial standings for 1st, 2nd, 3rd, and third-placed teams
    sorted_initial = group_goals_tracking.sort_values(by=['before_last_game_points', 'total_goal_difference', 'before_last_game_goals_scored'], ascending=[False, False, False])
    first_place_team = sorted_initial.iloc[0]['team'] if len(sorted_initial) > 0 else None
    second_place_team = sorted_initial.iloc[1]['team'] if len(sorted_initial) > 1 else None
    third_place_team = sorted_initial.iloc[2]['team'] if len(sorted_initial) > 2 else None

    # Identify third-place teams by group
    third_place_teams = []
    for stage_name, group_data in group_goals_tracking.groupby('stage'):
        sorted_group = group_data.sort_values(by=['before_last_game_points', 'total_goal_difference', 'before_last_game_goals_scored'], ascending=[False, False, False])
        if len(sorted_group) >= 3:
            third_place_teams.append(sorted_group.iloc[2]['team'])

    # Determine if the third-placed team is among the top 4 third-placed teams initially
    third_place_df = group_goals_tracking[group_goals_tracking['team'].isin(third_place_teams)]
    top4_third_place_teams = third_place_df.sort_values(
        by=['before_last_game_points', 'total_goal_difference', 'before_last_game_goals_scored'], 
        ascending=[False, False, False]
    ).head(4)['team'].tolist()
    is_top4_third_place_initial = 1 if third_place_team in top4_third_place_teams else 0

    # Step 2: Initialize composition tracking with initial composition (change_num = 0)
    initial_top_teams = set(sorted_initial.nsmallest(top_standings_limit, 'before_last_game_standing')['team'])
    composition_changes = [{
        'year': year,
        'stage': stage,
        'change_num': 0,
        'goal_time': 'initial',
        'home_team': None,
        'away_team': None,
        'scorer_nationality': None,
        'new_top_teams': list(initial_top_teams),
        'third_place_teams_list': third_place_teams,
        'top4_third_place': is_top4_third_place_initial,
        '1st': first_place_team,
        '2nd': second_place_team,
        '3rd': third_place_team
    }]
    change_counter = 0  # Counter for the number of composition changes

    # Step 3: Sort goals by goal_minute
    group_goals_last_day = group_goals_last_day.sort_values(by=['goal_minute'])

    # Step 4: Iterate through each goal and track changes in composition
    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home_team']
        away_team = goal['away_team']
        scorer_nationality = goal['scorer_nationality']

        # Update scores based on who scored the goal
        if scorer_nationality == home_team:
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
        elif scorer_nationality == away_team:
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1

        # Update total goals, goal difference, and total points
        group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored'] + group_goals_tracking['last_game_goals_scored']
        group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded'] + group_goals_tracking['last_game_goals_conceded']
        group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Calculate points for each team dynamically
        for i, row in group_goals_tracking.iterrows():
            if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 3 if year > 1994 else 2
            elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 1
            else:
                group_goals_tracking.loc[i, 'last_game_points'] = 0

        # Update total points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']

        # Sort teams to determine the current top standings
        group_goals_tracking = group_goals_tracking.sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored'], ascending=[False, False, False])
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1

        # Track teams in 1st, 2nd, and 3rd positions in the group
        top_teams = group_goals_tracking[group_goals_tracking['last_game_standing'] <= 3]
        first_place_team = top_teams.iloc[0]['team'] if len(top_teams) > 0 else None
        second_place_team = top_teams.iloc[1]['team'] if len(top_teams) > 1 else None
        third_place_team = top_teams.iloc[2]['team'] if len(top_teams) > 2 else None

        # Identify third-place teams by group after each goal
        third_place_teams = []
        for stage_name, group_data in group_goals_tracking.groupby('stage'):
            sorted_group = group_data.sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored'], ascending=[False, False, False])
            if len(sorted_group) >= 3:
                third_place_teams.append(sorted_group.iloc[2]['team'])

        # Determine the best four third-placed teams across groups
        third_place_df = group_goals_tracking[group_goals_tracking['team'].isin(third_place_teams)]
        top4_third_place_teams = third_place_df.sort_values(
            by=['total_points', 'total_goal_difference', 'total_goals_scored'], 
            ascending=[False, False, False]
        ).head(4)['team'].tolist()

        # Check if the third-placed team belongs to the top 4
        is_top4_third_place = 1 if third_place_team in top4_third_place_teams else 0

        # Track the current top teams
        current_top_teams = set(group_goals_tracking.nsmallest(top_standings_limit, 'last_game_standing')['team'])

        # Check if there is a change in composition
        if current_top_teams != initial_top_teams:
            change_counter += 1
            initial_top_teams = current_top_teams  # Update to new top composition
            composition_changes.append({
                'year': year,
                'stage': stage,
                'change_num': change_counter,
                'goal_time': goal['goal_minute'],
                'home_team': home_team,
                'away_team': away_team,
                'scorer_nationality': scorer_nationality,
                'new_top_teams': list(current_top_teams),
                'third_place_teams_list': third_place_teams,  # Third-placed teams in each group after the goal
                'top4_third_place': is_top4_third_place,      # 1 if third team in this group is in the top 4
                '1st': first_place_team,
                '2nd': second_place_team,
                '3rd': third_place_team
            })

    # Return the composition changes as a DataFrame
    composition_changes_df = pd.DataFrame(composition_changes)
    return composition_changes_df


In [47]:
# Initialize an empty list to store the results for each pair
all_composition_changes = []

# Get unique pairs of year and stage
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    composition_changes_df = track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_composition_changes.append(composition_changes_df)

# Concatenate all the results into a single DataFrame
final_composition_changes_df = pd.concat(all_composition_changes, ignore_index=True)

# Display the final DataFrame
display(final_composition_changes_df)


,year,stage,change_num,goal_time,home_team,away_team,scorer_nationality,new_top_teams,third_place_teams_list,top4_third_place,1st,2nd,3rd
0,2018.0,Group A,0,initial,None,None,None,"[Russia, Uruguay]",[Egypt],1,Russia,Uruguay,Egypt
1,2018.0,Group B,0,initial,None,None,None,"[Spain, Portugal]",[Iran],1,Portugal,Spain,Iran
2,2018.0,Group C,0,initial,None,None,None,"[Denmark, France]",[Australia],1,France,Denmark,Australia
3,2018.0,Group D,0,initial,None,None,None,"[Nigeria, Croatia]",[Iceland],1,Croatia,Nigeria,Iceland
4,2018.0,Group D,1,14,Nigeria,Argentina,Argentina,"[Argentina, Croatia]",[Nigeria],1,Croatia,Argentina,Nigeria
5,2018.0,Group D,2,51,Nigeria,Argentina,Nigeria,"[Nigeria, Croatia]",[Iceland],1,Croatia,Nigeria,Iceland
6,2018.0,Group D,3,86,Nigeria,Argentina,Argentina,"[Argentina, Croatia]",[Nigeria],1,Croatia,Argentina,Nigeria
7,2018.0,Group E,0,initial,None,None,None,"[Brazil, Switzerland]",[Serbia],1,Brazil,Switzerland,Serbia
8,2018.0,Group F,0,initial,None,None,None,"[Germany, Mexico]",[Sweden],1,Mexico,Germany,Sweden
9,2018.0,Group F,1,50,Mexico,Sweden,Sweden,"[Mexico, Sweden]",[Germany],1,Mexico,Sweden,Germany


In [48]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\standings_wc_fifa_wiki.xlsx'
final_composition_changes_df.to_excel(file_path, index=False)
